# Extracting the pulser position for each channel from the physics hit file
Jita (21 Jan 2024)

Due to low statistics for the physics files, it should be possible to extract the position of the pulser for each channel, byt looking at the signal that comes most frequently.

In [1]:
# Loading hit files
def load_hit(file):
    return lh5.load_dfs(file, ['cuspEmax_ctc_cal', 'is_valid_cal'], f"ch{ch}/hit")

In [2]:
def pd_run_extractor(string):
    try:
        start = string.index("l200-") + len("l200-")
        stop = string.index("-phy-")
        
    except Exception:
        print("Give a physics file")
        
    return string[start:stop]

#### Load the packages

In [3]:
# Import packages
import pygama.lgdo.lh5_store as lh5 
from lgdo import LH5Store
from legendmeta import LegendMetadata

import glob

from datetime import datetime, timezone

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
from scipy import optimize
from scipy.optimize import curve_fit

import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import ticker

import h5py
import numpy as np
import pandas as pd

#### Find and list all the files that are available in a period_run
This needs to be sorted!

In [4]:
# Selecting the data
expt = "l200"
data_type = "phy"
periods = ["p07"]
runs = []

pht_path = "/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht"

# Append runs to the array
for data_period in periods:
    runs_temp = sorted(glob.glob(f"{pht_path}/{data_type}/{data_period}/*"))
    runs.append([x[-4:] for x in runs_temp])
    print(runs)

[['r001', 'r002', 'r003', 'r004', 'r005', 'r006', 'r007']]


In [5]:
# Remove problematic runs from the array

for data_period, xy in zip(periods, runs):
    runs_to_remove = []  # To store indices of runs to be removed

    for i, data_run in enumerate(xy):
        hit_file = sorted(glob.glob(f"{pht_path}/{data_type}/{data_period}/{data_run}/{expt}-{data_period}-{data_run}-{data_type}-*-tier_pht.lh5"))
        
        if not (hit_file):
            print(f"Issue: Files not found for {data_period}, {data_run}")
            runs_to_remove.append(i)

    # Remove runs with issues
    for index in reversed(runs_to_remove):
        xy.pop(index)

print("Updated runs:", runs)

Updated runs: [['r001', 'r002', 'r003', 'r004', 'r005', 'r006', 'r007']]


In [7]:
affected_ch = {}

for data_period,xy in zip(periods, runs):
    for data_run in xy:
        hit_file = sorted(glob.glob(f"/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/{data_type}/{data_period}/{data_run}/{expt}-{data_period}-{data_run}-{data_type}-*-tier_pht.lh5"))
        
        print(hit_file[0])
        
        runid = pd_run_extractor(hit_file[0])
        print(runid)
        
        # extracting the datetime stamp
        try:
            dt = hit_file[0][hit_file[0].index("-cal-") + len("-cal-"):hit_file[0].index("-tier_")]
        except:
            dt = hit_file[0][hit_file[0].index("-phy-") + len("-phy-"):hit_file[0].index("-tier_")]

        # Making a dictionary for each of the systems
        lmeta = LegendMetadata()
        chmap = lmeta.hardware.configuration.channelmaps.on(dt)
        geds_dict = {}
        sipms_dict={}
        auxs_dict={}

        for channel_name, channel_data in chmap.items():
            if (channel_data.system == "geds"):
                geds_dict[channel_data["daq"]["rawid"]] = (channel_name,\
                                                           channel_data.location.string,\
                                                           channel_data.location.position,\
                                                           lmeta.channelmap(dt)[channel_name].analysis.usability,\
                                                           lmeta.channelmap(dt)[channel_name].analysis.psd_status['A/E'],\
                                                           lmeta.channelmap(dt)[channel_name].type,\
                                                           lmeta.channelmap(dt)[channel_name].production['mass_in_g'])

        geds_on = {}

        for x in geds_dict:
            if (geds_dict[x][3] == 'on'):
                geds_on[x] = (geds_dict[x][0], geds_dict[x][1], geds_dict[x][2], geds_dict[x][3], geds_dict[x][4], geds_dict[x][5])
        geds_df = pd.DataFrame.from_dict(geds_dict, orient='index', columns=['channel_name', 'string', 'posn', 'analysis', 'psd_flag', 'type', 'mass'])

        data = {}
        filtered_data = {}
        pulser = {}
        for ch in list(geds_on.keys()):
            # Load only the first file
            data[ch] = load_hit(hit_file[0])
            filtered_data[ch] = data[ch][data [ch] ["cuspEmax_ctc_cal"]> 400]
            #display(filtered_data[ch])
            pulser [ch] = filtered_data[ch]["cuspEmax_ctc_cal"].mean()
        pulser_df = pd.DataFrame.from_dict(list(pulser.items()))
        pulser_df.columns = ["channel", "pulser_keV"]
        pulser_df = pulser_df.set_index("channel")
        # Inner merge based on index
        merged_df = pd.merge(geds_df, pulser_df, left_index=True, right_index=True, how='inner')
        affected_ch[runid] = merged_df[(merged_df["pulser_keV"] >= 1150) & (merged_df["pulser_keV"] <= 1350)]
        display(merged_df[(merged_df["pulser_keV"] >= 1150) & (merged_df["pulser_keV"] <= 1350)])

/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r001/l200-p07-r001-phy-20230807T184254Z-tier_pht.lh5
p07-r001


,channel_name,string,posn,analysis,psd_flag,type,mass,pulser_keV
1104002,V05261B,1,3,on,valid,icpc,2393.0,1187.302501
1104004,V05266B,1,5,on,valid,icpc,1988.0,1168.607209
1107205,C00ANG3,2,4,on,missing,coax,2399.0,1197.272987
1108800,C00ANG5,2,5,on,missing,coax,2748.0,1195.524315
1108801,C00ANG2,2,6,on,missing,coax,2838.0,1242.316960
1110404,P00574B,3,3,on,present,ppc,778.9,1339.132577
1112000,P00698A,3,5,on,present,ppc,962.0,1261.578241
1112002,P00909C,3,7,on,present,ppc,882.9,1245.624392
1112003,P00661B,3,8,on,present,ppc,749.8,1256.233017
1121605,V05268A,8,9,on,valid,icpc,2298.0,1212.990561


/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r002/l200-p07-r002-phy-20230814T154547Z-tier_pht.lh5
p07-r002


,channel_name,string,posn,analysis,psd_flag,type,mass,pulser_keV
1104002,V05261B,1,3,on,valid,icpc,2393.0,1181.822143
1104004,V05266B,1,5,on,valid,icpc,1988.0,1159.854184
1107205,C00ANG3,2,4,on,missing,coax,2399.0,1179.718770
1108800,C00ANG5,2,5,on,missing,coax,2748.0,1194.710353
1108801,C00ANG2,2,6,on,missing,coax,2838.0,1241.932380
1110404,P00574B,3,3,on,present,ppc,778.9,1338.934348
1112000,P00698A,3,5,on,present,ppc,962.0,1259.947874
1112002,P00909C,3,7,on,present,ppc,882.9,1245.614525
1112003,P00661B,3,8,on,present,ppc,749.8,1256.211708
1121605,V05268A,8,9,on,valid,icpc,2298.0,1208.045547


/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r003/l200-p07-r003-phy-20230821T180340Z-tier_pht.lh5
p07-r003


,channel_name,string,posn,analysis,psd_flag,type,mass,pulser_keV
1104002,V05261B,1,3,on,valid,icpc,2393.0,1179.287595
1104004,V05266B,1,5,on,valid,icpc,1988.0,1168.658249
1107205,C00ANG3,2,4,on,missing,coax,2399.0,1190.283771
1108800,C00ANG5,2,5,on,missing,coax,2748.0,1194.846309
1108801,C00ANG2,2,6,on,missing,coax,2838.0,1241.507792
1110404,P00574B,3,3,on,present,ppc,778.9,1339.043733
1112000,P00698A,3,5,on,present,ppc,962.0,1264.599463
1112002,P00909C,3,7,on,present,ppc,882.9,1245.218401
1112003,P00661B,3,8,on,present,ppc,749.8,1256.144027
1121605,V05268A,8,9,on,valid,icpc,2298.0,1212.899876


/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r004/l200-p07-r004-phy-20230828T154920Z-tier_pht.lh5
p07-r004


,channel_name,string,posn,analysis,psd_flag,type,mass,pulser_keV
1104002,V05261B,1,3,on,valid,icpc,2393.0,1185.801095
1104004,V05266B,1,5,on,valid,icpc,1988.0,1168.676447
1107205,C00ANG3,2,4,on,missing,coax,2399.0,1214.575591
1108800,C00ANG5,2,5,on,missing,coax,2748.0,1193.929072
1108801,C00ANG2,2,6,on,missing,coax,2838.0,1241.385029
1110404,P00574B,3,3,on,present,ppc,778.9,1319.158520
1112000,P00698A,3,5,on,present,ppc,962.0,1266.317777
1112002,P00909C,3,7,on,present,ppc,882.9,1245.824756
1112003,P00661B,3,8,on,present,ppc,749.8,1238.123317
1121600,V00048B,8,4,on,valid,icpc,1815.8,1151.560822


/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r005/l200-p07-r005-phy-20230904T190210Z-tier_pht.lh5
p07-r005


OSError: [Errno 5] Unable to open file (file read failed: time = Tue Jan 23 15:04:45 2024
, filename = '/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/pht/phy/p07/r005/l200-p07-r005-phy-20230904T190210Z-tier_pht.lh5', file descriptor = 62, errno = 5, error message = 'Input/output error', buf = 0x7ffe939b2c80, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)